In [18]:
# make the matrix
# call simpledrift to fill in the values for different drift and bh
# pass on to heatmap
# pay attention to axes so they give the correct values
import numpy as np
import matplotlib.pyplot as plt
import simpledrift as sd
import math
from joblib import Parallel, delayed
import os
import csv
import imp

def csvgraph(driftvariance, betadvantage):

    csvfile = csv.reader(open('Matrices/da'+ str(driftvariance)+'ba'+ str(betadvantage)+'.csv','r'))
    csvparameter = csv.reader(open('Matrices/da'+ str(driftvariance)+'ba'+ str(betadvantage)+'parameters.csv','r'))
    row_count = sum(1 for row in csvfile)-2
    print(row_count)
    matrix=np.zeros((row_count, 2))
    for r in range(0,row_count):
        matrix[r,1]=
        matrix[r,2]=
        
    print(matrix)


#     # figuresavepath='figs'
#     # prefvariance=np.linspace(bhlower, bhupper, bhinterval)
#     # driftvariance=np.linspace(driftlower, driftupper, driftinterval)

#     numruns=bhinterval*driftinterval

#     #testing values are right
#     # for n in range(numruns):
#     #     print('driftvariance'+str(driftvariance[math.floor(n/bhinterval)])+ 'prefvariance'+ str(prefvariance[n%bhinterval]))

#     flatmatrix=np.zeros(numruns)

#     # for x in range(bhinterval):
#     flatmatrix[:]=Parallel(n_jobs=-1, verbose=10)(delayed(matrix for n in range(numruns)))
#             # print(["Drift is: ", driftvariance[y], "Bet-hedging is: ", driftvariance[x]] )
#             # print(matrix)
#     print(flatmatrix)

#     # flatmatrix=np.matrix(flatmatrix)
#     # flatmatrix=
#     matrix=np.zeros((bhinterval,driftinterval))
#     matrix[:,:]=flatmatrix.reshape((bhinterval, driftinterval), order='F')
#     matrixlog=np.log(matrix)
#     print(matrixlog)
#     print(matrix)



#     bhmargin=(bhupper-bhlower)/(bhinterval-1)/2
#     driftmargin=(driftupper-driftlower)/(driftinterval-1)/2
#     prefvariancemesh=np.linspace(bhlower-bhmargin, bhupper+bhmargin, bhinterval+1)
#     driftvariancemesh=np.linspace(driftlower-driftmargin, driftupper+driftmargin, driftinterval+1)
#     print(prefvariancemesh)
#     print(driftvariancemesh)

# # driftvariancegrid2, prefvariancegrid2= np.meshgrid(driftvariance, prefvariance)

#     # print(matrix)

#     fig,ax=plt.subplots()
#     scale=max(-np.min(matrixlog),np.max(matrixlog))
#     c=ax.pcolormesh(driftvariancemesh, prefvariancemesh, matrixlog, shading='flat', cmap='RdBu',  vmin=-scale, vmax=scale)


#     # c=ax.pcolormesh(driftvariancemesh, prefvariancemesh, matrixlog, shading='flat', cmap='RdBu',  vmin=-20, vmax=20)
#     ax.set_xlabel('Drift')
#     ax.set_ylabel('Bet-Hedging')
#     fig.colorbar(c, ax=ax)
#     ax.set_title('Log of Final Population')
#     plt.show

#     if os.path.exists(figuresavepath):
#         fig.savefig(os.path.join(figuresavepath,'heatmap.png'),bbox_inches='tight', pad_inches=.3)
#         print(figuresavepath)
#     else:
#         print('not saving, no valid path')

#     return matrix, driftvariance, prefvariance

In [19]:
csvgraph(.1,0.0)

10
[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]


In [6]:
import csv
import numpy as np
import matplotlib.pyplot as plt
import simpledrift as sd
import math
from joblib import Parallel, delayed
import os

q=0.2

csvfile = csv.reader(open('Matrices/driftadvantage' + str(q)+ '.csv','r'))
newfile = open('trial.txt','w')

readmatrix = np.genfromtxt('Matrices/driftadvantage' + str(q)+ '.csv')
print(readmatrix)
newfile.write(str(row))

for row in csvfile:
    hi = np.array((row))
    print(hi[0])
    newfile.write(str(row))
#     if i==t:
#         print(t)
#         place = t[4]
#         print(place)
#         outputfile.write(place+'\n')
#     i+=1

[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan]
# Each row is one day
#  Drift Advantage
0.000000000000000000e+00
-1.974787622049690716e-01
-8.950195753404055488e-02
-5.096576596937713033e-02
-3.124632852176570386e-02
-1.965608688431563777e-02
-1.251226622010306877e-02
-8.018810770679604683e-03
-5.166615562706578652e-03
-3.338938464075732332e-03
-2.162490818722582336e-03
-1.400505364550497234e-03
2.935543014646146975e-02
2.120993038258411900e-03
-1.782311397064453851e-02
-2.752993435035233638e-02
-3.012185714517626178e-02
-2.827538404610496769e-02
-2.389096030703155918e-02
-1.852825494986108656e-02
-1.335883983518583451e-02
-9.078223